In [1]:
import os

import numpy as np
from sklearn.utils.class_weight import compute_class_weight

from src.ppi import PPI

In [2]:
def get_project_rootpath():
    """
    获取项目根目录。此函数的能力体现在，不论当前module被import到任何位置，都可以正确获取项目根目录
    :return:
    """
    path = os.path.realpath(os.curdir)
    while True:
        for subpath in os.listdir(path):
            # PyCharm项目中，'.idea'是必然存在的，且名称唯一
            if '.idea' in subpath:
                return path
        path = os.path.dirname(path)

In [3]:
os.chdir(get_project_rootpath())
EMB_ROOT_PATH = 'data/emb/'
PPI_PATH = 'data/network/PPI-Network.txt'
DEEPWALK_EMB_ROOT_PATH = os.path.join(EMB_ROOT_PATH, 'deepwalk')
GENECOUNT_PATH = 'data/genecount.xls'
ppi = PPI(ppi_network_path=PPI_PATH, gene_count_path=GENECOUNT_PATH, k=6)

In [4]:
def dataloader(file_name, positive_gene: set, risklevel: dict):
    # file_name: emb files
    # positive_gene: 正例的gene
    # risklevel 出现了多少次的gene为positive gene

    data = file_name.strip('.emb').split('_')
    param = {i[0]: i[1:] for i in data[1:]}

    # 训练特征和label
    file_path = os.path.join(DEEPWALK_EMB_ROOT_PATH, file_name)
    with open(file_path, 'r') as f:
        data = [line.strip().split() for line in f.readlines()[1:]]
    X = [line[1:] for line in data]
    target = [1 if int(line[0]) in positive_gene else 0 for line in data]
    X = np.asarray(X, dtype=float)
    target = np.asarray(target, dtype=int)

    # 权重
    class_weight = compute_class_weight(class_weight='balanced', classes=[0, 1], y=target)
    sample_weight = [risklevel[int(line[0])] * class_weight[1]
                     if int(line[0]) in positive_gene else class_weight[0] for line in data]
    return param, X, target, sample_weight

In [5]:
from sklearn.model_selection import train_test_split

dataset = os.listdir(DEEPWALK_EMB_ROOT_PATH)[0]
params, X, y, weight = dataloader(dataset, ppi.positive_id, ppi.risk_level)
X_train, X_test, y_train, y_test, weight_train, weights_test = train_test_split(X, y,
                                                                                weight,
                                                                                test_size=0.33,
                                                                                random_state=420)

参数分为：
1. 通用参数：宏观函数控制
2. Booster参数：控制每一步的booster。一般可以调控模型的效果和计算代价。调参主要是调这个
3. 学习目标参数：控制训练目标的表现。对问题的划分主要体现在学习目标参数上。比如是分类还是回归，二分类还是多分类
# 通用参数
1. booster: gbtree和gblinear, 前者是树结构, 后者是线性模型
2. silent: 静默模式
3. nthread: 线程数, 一般设置为-1.
# Booster参数
1. n_estimator/num_boosting_rounds: 生成的最大树的数目/最大的迭代次数
2. learning_rate/eta: 每一步迭代的步长, 很重要. 太大会影响准确率, 太小了慢. 默认值是0.3,一般使用比默认值小一点.
3. gamma: 默认0.
4. subsample: 默认为1. 对每棵树, 随机采样的比例. 减小这个参数的值, 算法会更加保守避免过拟合.
5. colsample_bytree: 默认值为1. 控制每棵树随机采样的列数的占比(每一列是一个特征). 典型值: 0.5-1
6. colsample_blevel: 默认值为1. 比前一个参数更加细致, 限制到每棵树每次节点分裂的时候采样的比例.
7. max_depth: 树的最大深度, 用于控制过拟合. max_depth越大, 模型学习的更加具体. 0表示没有限制.
8. max_delta_step: 默认0, 常用0. 限制每棵树权重改变的最大步长, 设置为0时,表示没有限制. 通常不用设置,当类别不平衡时可以设置来优化逻辑回归.
9. lambda/reg_lambda: 默认值为0. 权重的L2正则化项
10. alpha/reg_alpha: 默认值为0. 权重的L1正则化项. 可以应用在很高纬度的情况下,使得算法的速度更快.
11. scale_pos_weight: 默认值为1. 类别不平衡的时候, 把这个参数设定为正值, 可以使得算法收敛的更快. 通常可以设置为 负样本/正样本.
# 学习目标参数
## objective [缺省值:`reg:linear`]
- `reg:linear`
- `reg:logistic`
- `binary:logistic`
- `binary:logitraw`
- `count:poisson`
- `multi:softmax`
- `multi:softprob`
## eval_metric
- `rmse`
- `mae`
- `logloss`
- `error`
- `merror`
- `mlogloss`
- `aur`
- `ndcg`
- `map`

In [18]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

cv_params = {
    'n_estimators': [100, 300, 500, 700, 1000, 1250, 1500, 2000],
    'learning_rate': [0.01, 0.015, 0.025, 0.05, 0.1, 0.15, 0.3]
}

other_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_depth': 5,
    'gamma': 0,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'n_estimators': 500,
    'min_child_weight': 1,
    'seed': 420
}
model = xgb.XGBClassifier(**other_params)

optimized_GBM = GridSearchCV(estimator=model,
                             param_grid=cv_params,
                             scoring=['f1', 'roc_auc', 'accuracy'],
                             refit='f1',
                             cv=5,
                             verbose=1,
                             n_jobs=-1)

optimized_GBM.fit(X_train, y_train)
evaluate_result = optimized_GBM.cv_results_

print(f'每轮迭代运行结果:{evaluate_result}')
print(f'参数的最佳取值：{optimized_GBM.best_params_}')
print(f'最佳模型得分:{optimized_GBM.best_score_}')

Fitting 5 folds for each of 56 candidates, totalling 280 fits
每轮迭代运行结果:{'mean_fit_time': array([ 17.74601002,  51.27845716,  85.26572733, 119.83725209,
       173.15632863, 213.22183075, 253.45590014, 338.05404773,
        17.01271324,  51.07575645,  85.38925271, 121.3078342 ,
       179.81939969, 222.54636316, 262.53371081, 352.58508849,
        19.42110138,  52.3026413 ,  88.30076733, 117.56416473,
       164.61917605, 199.45117803, 239.43744106, 322.78921862,
        16.19851508,  48.2603301 ,  82.32796278, 112.62923241,
       159.29818697, 200.82155418, 257.52113395, 359.27667675,
        19.425038  ,  58.16639419,  87.22291536, 118.62989798,
       167.72136059, 209.60220418, 252.18174958, 323.50170755,
        16.15195374,  49.05451522,  82.29930801, 115.42684941,
       166.96129575, 204.56666751, 240.41661992, 314.22740183,
        16.85727596,  49.44343314,  80.81200271, 110.63475022,
       155.69727907, 188.66003265, 218.8469234 , 233.10204082]), 'std_fit_time': array([ 0.1

In [8]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

cv_params = {
    'n_estimators': [1250, 1350, 1400, 1450, 1500, 1550, 1600, 1750],
    'learning_rate': [0.005, 0.01, 0.015, 0.02, 0.05]
}

other_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_depth': 5,
    'gamma': 0,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'n_estimators': 500,
    'min_child_weight': 1,
    'seed': 420
}
model = xgb.XGBClassifier(**other_params)

optimized_GBM = GridSearchCV(estimator=model,
                             param_grid=cv_params,
                             scoring=['f1', 'roc_auc', 'accuracy'],
                             refit='f1',
                             cv=5,
                             verbose=1,
                             n_jobs=-1)

optimized_GBM.fit(X_train, y_train)
evaluate_result = optimized_GBM.cv_results_

print(f'每轮迭代运行结果:{evaluate_result}')
print(f'参数的最佳取值：{optimized_GBM.best_params_}')
print(f'最佳模型得分:{optimized_GBM.best_score_}')

Fitting 5 folds for each of 40 candidates, totalling 200 fits
每轮迭代运行结果:{'mean_fit_time': array([323.30145183, 345.29886794, 349.30411606, 327.51547008,
       328.10244913, 338.16405177, 347.03652787, 378.98366928,
       269.89271889, 287.82463579, 296.82184968, 312.12482862,
       324.59863553, 335.52693563, 343.75447187, 372.23105302,
       267.81134715, 288.32670841, 297.16123495, 307.31380515,
       318.03278809, 328.09527965, 337.21407094, 366.70401897,
       261.1788352 , 280.31157856, 286.54261346, 296.27362237,
       306.65656352, 316.84244471, 327.33064494, 357.92859364,
       255.92738509, 275.90841784, 287.13846936, 296.30436006,
       305.74661407, 315.75905023, 327.18465791, 308.57323356]), 'std_fit_time': array([ 0.6965268 ,  5.30800895,  1.70785705, 16.35457139,  1.10635861,
        0.64766564,  1.50522912,  1.23106237,  1.35037445,  1.97536083,
        0.44107419,  1.70785523,  0.97918219,  1.03687692,  2.84851111,
        0.4924795 ,  0.45481676,  1.22131421,  

In [ ]:
# 然后调试minchildweight和max_depth
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


cv_params = {
    'max_depth': [3,4,5,6,7,8,9,10],
    'min_child_weight': [1,2,3,4,5,6],
}

other_params = {
    'n_estimators': 1550,
    'learning_rate': 0.01
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_depth': 5,
    'gamma': 0,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'n_estimators': 500,
    'min_child_weight': 1,
    'seed': 420
}
model = xgb.XGBClassifier(**other_params)

optimized_GBM = GridSearchCV(estimator=model,
                             param_grid=cv_params,
                             scoring=['f1', 'roc_auc', 'accuracy'],
                             refit='f1',
                             cv=5,
                             verbose=1,
                             n_jobs=-1)

optimized_GBM.fit(X_train, y_train)
evaluate_result = optimized_GBM.cv_results_

print(f'每轮迭代运行结果:{evaluate_result}')
print(f'参数的最佳取值：{optimized_GBM.best_params_}')
print(f'最佳模型得分:{optimized_GBM.best_score_}')